In [3]:
import pandas as pd
import numpy as np
import time

#read data from data ingestor script
df = pd.read_csv('transactions.csv')

df.head()

,Unnamed: 0,country,customer_id,invoice,year,month,day,stream_id,times_viewed,price
0,0,United Kingdom,15581.0,493463,2018,1,1,20895,1,2.95
1,1,United Kingdom,15581.0,493463,2018,1,1,20866,1,1.25
2,2,United Kingdom,15581.0,493463,2018,1,1,85198,12,0.85
3,3,United Kingdom,15581.0,493463,2018,1,1,20897,1,2.95
4,4,United Kingdom,15581.0,493463,2018,1,1,20898,1,2.95


In [ ]:
'''
PRE PROCESSING

clean up Unnamed column DONE
transform dmy columns into 1 DONE
index on date DONE
save as new df/csv DONE

sum up total revenue for each country
sort, identify top 10
cut out data for top 10 countries
save as new df/csv

DATA VIZZES
data types
descriptive stats
distributions of each variable
visualise price over time

counts per country of:
- unique customer ids?
- invoices?
- unique stream ids?

'''

In [ ]:
#drop 'unnamed' column
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [11]:
#create data column. day, month and years to be concatenated into one value to be stored in this column
df['date'] = np.NaN
df['date']=df['year'].astype(str) + df['month'].astype(str).str.zfill(2)+ df['day'].astype(str).str.zfill(2)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

In [50]:
#group by date and year, aggregation = sum
dfdate = df.groupby(by='date').sum()
dfyear = df.groupby(by='year').sum()

#save csv file with index as the date
dfdate.to_csv('transactions_date_index.csv')

#dfmonth = df.groupby(by='month')
#dfday = df.groupby(by='day')


In [91]:
#group by country and find top 10 highest revenue countries
dfcountry = df.groupby(by='country').sum()
dfcountry['price'] = dfcountry['price'].round(2)
#top 10 countries with the highest revenue:
dfcountry.sort_values(by='price', ascending=False)[:10]

,customer_id,year,month,day,times_viewed,price
country,,,,,,
United Kingdom,8.770331e+09,1516160498,4967307,11294245,3725703,3521513.51
EIRE,1.781244e+08,27401294,89337,214425,111636,107069.21
Germany,1.683921e+08,26916270,81741,203006,112618,49271.82
France,1.245792e+08,19979533,62335,139287,83124,40565.14
Norway,1.002374e+07,1606587,7108,12753,7881,38494.75
Spain,3.335718e+07,5259924,15379,37596,19573,16040.99
Hong Kong,0.000000e+00,557168,1114,4173,2635,14452.57
Portugal,2.212917e+07,3663166,13032,27623,14195,13528.67
Singapore,3.950640e+06,625773,1702,3578,3042,13175.92
